In [1]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [2]:
import os
import random
import time
import itertools  
import numpy                 as np
import tensorflow            as tf
import tensorflow_addons     as tfa
import matplotlib.pyplot     as plt
import pandas                as pd
import gudhi                 as gd
import gudhi.representations as sktda
import sys

from difftda                              import *
from gudhi.representations.vector_methods import Atol as atol
from gudhi.representations.kernel_methods import SlicedWassersteinKernel as swk
from gudhi.wasserstein                    import wasserstein_distance
from gudhi.representations                import pairwise_persistence_diagram_distances as ppdd
from mpl_toolkits.mplot3d                 import Axes3D
from scipy.linalg                         import expm
from scipy.io                             import loadmat
from scipy.sparse                         import csgraph
from scipy.linalg                         import eigh
from sklearn.base                         import BaseEstimator, TransformerMixin
from sklearn.metrics                      import pairwise_distances, accuracy_score
from sklearn.manifold                     import MDS, LocallyLinearEmbedding, SpectralEmbedding
from sklearn.preprocessing                import MinMaxScaler, Normalizer, LabelEncoder
from sklearn.pipeline                     import Pipeline, FeatureUnion
from sklearn.svm                          import SVC
from sklearn.ensemble                     import RandomForestClassifier
from sklearn.neighbors                    import KNeighborsClassifier
from sklearn.model_selection              import GridSearchCV, KFold, StratifiedKFold
from sklearn.cluster                      import KMeans

/Users/nyckeesluca/opt/anaconda3/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.5.0 and strictly below 2.8.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.3.1 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [8]:
from tqdm import tqdm
import plotly.graph_objects as go

# Point cloud

In this section, we implement the point cloud experiment, where we optimize loops in a point cloud using 1-dimensional homology.

In [4]:
use_reg = 1

In [5]:
np.random.seed(1)
Xinit = np.array(np.random.uniform(high=1., low=-1., size=(300,2)), dtype=np.float32)

In [14]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=Xinit[:,0], y=Xinit[:,1],mode='markers'))
fig.show()

In [18]:
X = tf.Variable(initial_value=Xinit, trainable=True)
model = RipsModel(X=X, mel=12., dim=1, card=50)
lr = tf.keras.optimizers.schedules.InverseTimeDecay(initial_learning_rate=1e-1, decay_steps=10, decay_rate=.01)
optimizer = tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.)
sigma = 0.001

losses, dgms = [], []
for epoch in tqdm(range(50+1)):
    
    with tf.GradientTape() as tape:
        
        dgm = model.call()
        if use_reg:
            loss = -tf.math.reduce_sum(tf.square(.5*(dgm[:,1]-dgm[:,0]))) + tf.reduce_sum(tf.maximum(tf.abs(X)-1, 0))
        else:
            loss = -tf.math.reduce_sum(tf.square(.5*(dgm[:,1]-dgm[:,0])))

    gradients = tape.gradient(loss, model.trainable_variables)
    np.random.seed(epoch)
    gradients[0] = gradients[0] + np.random.normal(loc=0., scale=sigma, size=gradients[0].shape)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    losses.append(loss.numpy())
    dgms.append(dgm)

100%|██████████| 51/51 [02:31<00:00,  2.97s/it]


In [19]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=model.X.numpy()[:,0], y=model.X.numpy()[:,1],mode='markers'))
fig.show()